# Использование LlamaQueryEngineRetriever вместе с Langchain
1. Устанавливаем нужные пакеты

In [ ]:
!pip install llama_index wikipedia gigachain openai
!pip uninstall -y langchain

2. Загружаем статью из Википедии для поиска по ней
3. Инициализируем VectorStoreIndex где будут храниться семантические данные по статье

In [46]:
import os

from llama_index import download_loader, VectorStoreIndex

os.environ["OPENAI_API_KEY"] = "<Ключ OpenAI для получения с их стороны ембедингов>"

WikipediaReader = download_loader("WikipediaReader")

loader = WikipediaReader()

documents = loader.load_data(
    pages=["Московский метрополитен"], auto_suggest=False, lang="ru"
)

index = VectorStoreIndex.from_documents(documents)

4. Инициализируем класс LlamaQueryEngineRetriever, который будет искать документы силами LLamaIndex

In [49]:
from langchain.retrievers import LlamaQueryEngineRetriever
from llama_index.response_synthesizers import ResponseMode, get_response_synthesizer

response_synthesizer = get_response_synthesizer(response_mode=ResponseMode.NO_TEXT)
query_engine = index.as_query_engine(
    response_synthesizer=response_synthesizer,
)
retriever = LlamaQueryEngineRetriever(
    query_engine=query_engine,
)

5. Инициализируем цепочку RetrievalQA, отвечающая на вопросы пользователя, исходя из документов, которые мы нашли

In [50]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import GigaChat

llm = GigaChat(
    # тут данные для входа в GigaChat
)

prompt_template = """Используя только контекст и не свои знания ниже дай ответ на вопрос (Question). Не используй свои знания только контекст.
КОНТЕКСТ
-----
{context}
-----
Question: {question}
AI:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": PROMPT},
)

6. Прогоняем вопрос

In [51]:
qa.run("Что такое метротур?")

'Метротур — это проект, организованный Городским экскурсионным бюро и Московским метрополитеном, который предоставляет возможность пройти трехдневное обучение и узнать больше о истории развития метро, его правилах и эксплуатации. В рамках программы "Метротур" проводятся экскурсии по публично доступным станциям метро.'